# 변이형 오토인코더(Variational AutoEncoder)
오토인코더는 잠재 공간의 값을 고정된 값(표현 벡터)으로 나타냈지만
변이형 오토인코더는 잠재 공간의 값을 가우시안 확률분포(정규분포) 값의 범위로 제공한다.
즉, 잠재 공간의 값은 평균과 분산으로 매핑된다.
이미지 데이터가 충분하지 않은 분야에서 사용

In [ ]:
from keras.datasets import mnist
from keras.models import Sequential,Model
from keras.layers import Dense, Conv2D, MaxPooling2D, UpSampling2D
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
(x_train, _),(x_test,_) = mnist.load_data()
x_train=x_train.reshape(x_train.shape[0], 28,28,1).astype('float32')/255
x_test=x_test.reshape(x_test.shape[0], 28,28,1).astype('float32')/255

autoencoder=Sequential()

# encoder : 모델은 고차원 입력 데이터를 저차원 표현벡터로 앙북(차원 축소, 특징추출)
autoencoder.add(Conv2D(32,kernel_size=3,padding='same',input_shape=(28,28,1),activation='relu'))
autoencoder.add(MaxPooling2D(pool_size=2,padding='same'))
autoencoder.add(Conv2D(16,kernel_size=3,padding='same',activation='relu'))
autoencoder.add(MaxPooling2D(pool_size=2,padding='same'))
autoencoder.add(Conv2D(8,kernel_size=3,padding='same',activation='relu'))

# decoder : 모델은 주어진 표현벡터를 원본차원 압축을 해제. (차원복구)
autoencoder.add(Conv2D(8,kernel_size=3,padding='same',activation='relu'))
autoencoder.add(UpSampling2D())
autoencoder.add(Conv2D(16,kernel_size=3,padding='same',activation='relu'))
autoencoder.add(UpSampling2D())
autoencoder.add(Conv2D(32,kernel_size=3,padding='same',activation='relu'))
autoencoder.add(Conv2D(1,kernel_size=3,padding='same',activation='sigmoid'))

print(autoencoder.summary())

In [ ]:
autoencoder.compile(optimizer='adam',loss='binary_crossentropy')
history = autoencoder.fit(x_train,x_train, epochs=30, batch_size=128, validation_data=(x_test,x_test),verbose=2)
print(history.history)


In [ ]:
random_test=np.random.randint(x_test.shape[0],size=5)
ae_imgs=autoencoder.predict(x_test)

# 시각화
plt.figure(figsize=(7,2))
for i, image_idx in enumerate(random_test):
    ax=plt.subplot(2,5,i+1)
    plt.imshow(x_test[image_idx].reshape(28,28))
    ax.axis('off')

    ax=plt.subplot(2,5,i+5+1)
    plt.imshow(ae_imgs[image_idx].reshape(28,28))
    ax.axis('off')
plt.show()

In [ ]:
plt.plot(history.history['loss'],c='r',label='loss')
plt.plot(history.history['val_loss'],c='b',label='val_loss')
plt.legend()
plt.show()